In [ ]:
import sys
sys.path.insert(0, "../../code-2023-deephyptrails/")  # you might need to use the absolute path here

In [ ]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import umap
import umap.plot
from sklearn.ensemble import RandomForestClassifier

from Code.Dataset.HypothesisDataset import HypothesisDataset
from Code.Dataset.SyntheticDataset import RFWalkDataset
from Code.Models.RandoLMForest import generate, get_probabilities_per_token

In [ ]:
model = pickle.load(
    open("code-2023-deephyptrails/data/all-data/randoLM_data-even.pkl", "rb")
)
model

In [ ]:
generate(model, initial_context=[0], max_length=10)

In [ ]:
dataset = HypothesisDataset("code-2023-deephyptrails/data/all-data/dataset.jsonl")

In [ ]:
for hypothesis, hypothesis_walks in dataset.hypothesis_walks.items():
    print(hypothesis, len(hypothesis_walks))

In [ ]:
def plot_stuff(hypo, walkies):
    ds = RFWalkDataset(walkies)
    probs = get_probabilities_per_token(model, ds.inputs, ds.targets)
    print(len(probs))
    probs_by_token = {i: [] for i in range(102)}
    for p, t in zip(probs, ds.targets):
        probs_by_token[t].append(p)

    probs_by_token = {i: np.mean(p) for i, p in probs_by_token.items()}
    plt.figure()
    plt.plot(probs_by_token.keys(), probs_by_token.values())
    plt.xlabel("Token")
    plt.ylabel("Probability")
    plt.title(hypo)


    # likeliest_tokens_ranked = results["logits"].argsort(2, descending=True)
    # ranks = (target_walkies.unsqueeze(-1) == likeliest_tokens_ranked).nonzero()

    # ranks_df = pd.DataFrame(ranks, columns=["sample", "step", "rank"])
    # ranks_df["target_token"] = target_walkies.flatten()
    # ranks_df["predicted_token"] = results["logits"].argmax(2).flatten()

    # if by == "decoding_step":
    #     ranks_df.groupby("step").mean()["rank"].plot(
    #         xticks=list(range(21)),
    #         label=' '.join(name.split("-")[1:]).title() + f" ({results['loss'].item():.2f})",
    #         xlabel="decoding time step",
    #         ylabel="avg. rank",
    #     )

In [ ]:
for name in ["hyp-even", "hyp-odd"]:
    print(name)
    plot_stuff(name, dataset.hypothesis_walks[name][:200])

In [ ]:
for hypothesis, hypothesis_walks in dataset.hypothesis_walks.items():
    ds = RFWalkDataset(hypothesis_walks)
    probs = get_probabilities_per_token(model, ds.inputs, ds.targets, vocab_size=100)
    probs_by_index = {i: [] for i in range(50)}
    for p, i in zip(probs, ds.indices):
        probs_by_index[i].append(p)

    probs_by_index = {i: np.mean(p) for i, p in probs_by_index.items()}

    plt.figure()
    plt.plot(probs_by_index.keys(), probs_by_index.values())
    plt.xlabel("Index")
    plt.ylabel("Probability")
    plt.title(hypothesis)